In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

order_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that determines whether the customer input message contains an order_id.
            If the input contains an order_id, respond with '조회 가능'.
            If the input does not contain an order_id, respond with '조회 불가능'.
            """
        ),
        ("human", "{input}")
    ]
)

model = ChatOpenAI()

order_cancel_chain = order_cancel_prompt | model 

In [4]:
order_cancel_chain.invoke({"input": "좀 전에 한 주문 취소하고 싶어"})

AIMessage(content='조회 불가능', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 83, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f33d576b-f0a2-410e-a441-f558a74a5552-0')

In [ ]:
RunnablePassthrough.assign(recent_orders=order_cancel_chain)

In [ ]:
def fetch_recent_orders(dict):
    user_id = dict["user_id"]
    try:
        orders = Order.objects.filter(user_id=user_id).order_by('-created_at')[:5]
        recent_orders = []
        for order in orders:
            order_items = OrderItem.objects.filter(order=order)
            items_details = [
                {
                    "product_name": item.product.product_name,
                    "quantity": item.quantity,
                    "price": float(item.price)  # Decimal을 float으로 변환
                } for item in order_items
            ]
            recent_orders.append({
                "id": order.id,
                "created_at": order.created_at.isoformat(),
                "order_status": order.order_status,
                "items": items_details
            })
        print("recent orders -> ", recent_orders)
        return recent_orders
    except ObjectDoesNotExist:
        return []

In [ ]:
def cancel_route_by_order_id(info):
    from .chains import order_chain
    print("cancel_route_by_order_id 함수로 전달된 데이터 -> ", info)
    if "조회 가능" in info["recent_orders"].content.lower():
        return RunnableLambda(cancel_order)
    else:
        return RunnableLambda(fetch_recent_orders)

주문 취소부터 작성 중이었는데 순간 헷갈

In [ ]:
from pydantic import BaseModel, Field, conint, condecimal
from typing import List, Optional
from langchain_core.output_parsers import PydanticOutputParser

class UpdateOrderData(BaseModel):
    order_id: int = Field(..., description="ID of the order to update")
    user_id: Optional[int] = Field(None, description="ID of the user updating the order")
    items: Optional[List[OrderItemData]] = Field(None, description="Updated list of items in the order")
    new_status: Optional[str] = Field("order_canceled", description="New status of the order")

update_order_parser = PydanticOutputParser(pydantic_object=UpdateOrderData)


In [ ]:
extract_cancel_args_prompt = PromptTemplate(
    template="""
    You are a robot designed to extract necessary parameters for processing each order request.
    Only include the extracted parameters in your response.

    Product Information:
    {products}

    User ID: {user_id}

    User Input: {input}

    Provide only the extracted parameters in the following format:
    {format_instructions}
    """,
    input_variables=["user_id", "input", "products"],
    partial_variables={"format_instructions": [_order_parser.get_format_instructions()]},
)

In [ ]:
extract_cancel_args_chain = extract_cancel_args_prompt | model | update_order_parser 

In [ ]:
extract_cancel_args_chain